### This notebook contains the code to generate the submission for the "Flu Shot Learning: Predict H1N1 and Seasonal Flu Vaccines" competition.

The submission should contain 3 columns, with the respondent_id, the probability someone gets the H1N1 vaccine (h1n1_vaccine), and the probability that someone gets the flu shot (seasonal_vaccine).

To train the model, we use the training_set_features.csv data, with the training_set_labels.csv data as the known probabilities. Finally we want to predict the values for the test_set_features.csv data.

The score is evaluated using the receiver operating characteristic curve (ROC AUC), with default "average='macro'".

In [1]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier

In [13]:
# Start by loading the data. X and y are already split in two datasets
X = pd.read_csv("training_set_features.csv")
y = pd.read_csv("training_set_labels.csv")
X.head()

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb


In [14]:
# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=42)

# For preprocessing, we'll fill in missing values and encode categorical variables
X_train_prep = X_train.copy()
X_valid_prep = X_valid.copy()

In [15]:
# Check for missing values
print(X.isnull().sum().max())
# See if the number of missing values is large compared to the total, to see if it could just be dropped
print(len(X))
# With 26707 rows and missing values up to 13470, we can't just drop the rows with missing values

13470
26707


In [16]:
# To fill in missing values, we differentiate between numerical and categorical columns
# We'll fill in missing values for numerical columns with the mean of the column
numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]
for col in numerical_cols:
    X_train_prep[col].fillna(X_train_prep[col].mean(), inplace=True)
    X_valid_prep[col].fillna(X_valid_prep[col].mean(), inplace=True)

# For categorical columns, we distinguish ordinal and nominal columns
# For the nominal, we add the "Unknown" category. For ordinal, we fill in with the mode (as unknown cannot be sorted in the ranking)
nominal_cols = ["race", "sex", "marital_status", "rent_or_own", "employment_status", "hhs_geo_region", 
                "census_msa", "employment_industry", "employment_occupation"]
for col in nominal_cols:
    X_train_prep[col].fillna("Unknown", inplace=True)
    X_valid_prep[col].fillna("Unknown", inplace=True)

ordinal_cols = [cname for cname in X_train.columns if (X_train[cname].dtype == "object" and cname not in nominal_cols)]
for col in ordinal_cols:
    X_train_prep[col].fillna(X_train_prep[col].mode()[0], inplace=True)
    X_valid_prep[col].fillna(X_valid_prep[col].mode()[0], inplace=True)

#print(X_train_prep.isnull().sum())
#print("\n")
#print(X_valid_prep.isnull().sum())
# Sort by respondent_id
X_train_prep = X_train_prep.sort_values("respondent_id")
X_valid_prep = X_valid_prep.sort_values("respondent_id")

C:\Users\Marco van der Ploeg\AppData\Local\Temp\ipykernel_19988\49310599.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train_prep[col].fillna(X_train_prep[col].mean(), inplace=True)
C:\Users\Marco van der Ploeg\AppData\Local\Temp\ipykernel_19988\49310599.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting v

In [17]:
# Then there are a number of columns that are categorical to deal with in preprocessing
# Start with ordinal ones, where a specific order is implied

# First age groups
#print(X_train_prep["age_group"].unique())
age_group_order = ["18 - 34 Years", "35 - 44 Years", "45 - 54 Years", "55 - 64 Years", "65+ Years"]
X_train_prep["age_group"] = pd.Categorical(X_train_prep["age_group"], categories=age_group_order, ordered=True)
X_valid_prep["age_group"] = pd.Categorical(X_valid_prep["age_group"], categories=age_group_order, ordered=True)

X_train_prep["age_group"] = X_train_prep["age_group"].cat.codes
X_valid_prep["age_group"] = X_valid_prep["age_group"].cat.codes
X_train_prep[["respondent_id", "age_group"]].head()

,respondent_id,age_group
0,0,3
1,1,1
2,2,0
3,3,4
4,4,2


In [18]:
# education
#print(X_train_prep["education"].unique())
edu_order = ["< 12 Years", "12 Years", "Some College", "College Graduate"]
X_train_prep["education"] = pd.Categorical(X_train_prep["education"], categories=edu_order, ordered=True)
X_valid_prep["education"] = pd.Categorical(X_valid_prep["education"], categories=edu_order, ordered=True)

X_train_prep["education"] = X_train_prep["education"].cat.codes
X_valid_prep["education"] = X_valid_prep["education"].cat.codes
X_train_prep[["respondent_id", "education"]].head()

,respondent_id,education
0,0,0
1,1,1
2,2,3
3,3,1
4,4,2


In [19]:
# income_poverty
#print(X_train_prep["income_poverty"].unique())
inc_order = ["Below Poverty", "<= $75,000, Above Poverty", "> $75,000"]
X_train_prep["income_poverty"] = pd.Categorical(X_train_prep["income_poverty"], categories=inc_order, ordered=True)
X_valid_prep["income_poverty"] = pd.Categorical(X_valid_prep["income_poverty"], categories=inc_order, ordered=True)

X_train_prep["income_poverty"] = X_train_prep["income_poverty"].cat.codes
X_valid_prep["income_poverty"] = X_valid_prep["income_poverty"].cat.codes
X_train_prep[["respondent_id", "income_poverty"]].head()

,respondent_id,income_poverty
0,0,0
1,1,0
2,2,1
3,3,0
4,4,1


In [20]:
# Next are nominal columns, where no order is implied. We check if there are not too many unique values, and one-hot encode them if so
#for col in nominal_cols:
#    print(col, X_train_prep[col].nunique())

# At most 23, so we can one-hot encode them
X_train_prep = pd.get_dummies(X_train_prep, columns=nominal_cols, dtype=int)
X_valid_prep = pd.get_dummies(X_valid_prep, columns=nominal_cols, dtype=int)
X_train_prep.head()

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,employment_occupation_qxajmpny,employment_occupation_rcertsgn,employment_occupation_tfqavkke,employment_occupation_ukymxvdu,employment_occupation_uqqtjvyb,employment_occupation_vlluhbov,employment_occupation_xgwztkwe,employment_occupation_xqwwgdyp,employment_occupation_xtkaffoo,employment_occupation_xzmlyyjv
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,0,0,0,0,0,0,1,0,0,0
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# Sort y by respondent_id and then drop the column
y_train = y_train.sort_values("respondent_id")
y_valid = y_valid.sort_values("respondent_id")
y_train = y_train.drop("respondent_id", axis=1)
y_valid = y_valid.drop("respondent_id", axis=1)
y_train.head()

,h1n1_vaccine,seasonal_vaccine
0,0,0
1,0,1
2,0,0
3,0,1
4,0,0


In [22]:
# Now we can train a model
model = XGBClassifier(n_estimators=1000, learning_rate=0.05, n_jobs=-1, early_stopping_rounds=5, random_state=42)
model.fit(X_train_prep, y_train,
          eval_set=[(X_valid_prep, y_valid)],
          verbose=False)
y_pred = model.predict_proba(X_valid_prep)
print(y_pred)
y_pred_h1n1 = y_pred[:, 0]
y_pred_seas = y_pred[:, 1]
print(y_pred_h1n1)

score = roc_auc_score(y_valid, y_pred, average='macro')
score_h1n1 = roc_auc_score(y_valid["h1n1_vaccine"], y_pred_h1n1)
score_seas = roc_auc_score(y_valid["seasonal_vaccine"], y_pred_seas)
print(score, score_h1n1, score_seas)

[[0.02284514 0.15939434]
 [0.16789395 0.38940513]
 [0.42000243 0.5157307 ]
 ...
 [0.15883364 0.65428597]
 [0.5079425  0.8405155 ]
 [0.01895313 0.12672555]]
[0.02284514 0.16789395 0.42000243 ... 0.15883364 0.5079425  0.01895313]
0.8674621491892098 0.8700400877392027 0.864884210639217


In [23]:
# Optimise hyperparameters; using GridSearchCV does not work well
param_grid = {
    'n_estimators': [500, 1000, 1200],
    'learning_rate': [0.01, 0.05, 0.1]
}

scores = []
for n_est in param_grid["n_estimators"]:
    for lr in param_grid["learning_rate"]:
        model = XGBClassifier(n_estimators=n_est, learning_rate=lr, n_jobs=-1, early_stopping_rounds=5, random_state=42)
        model.fit(X_train_prep, y_train,
                  eval_set=[(X_valid_prep, y_valid)],
                  verbose=False)
        y_pred = model.predict_proba(X_valid_prep)
        score = roc_auc_score(y_valid, y_pred, average='macro')
        print(n_est, lr, score)
        scores.append(score)
print(max(scores))

500 0.01 0.8667736761949809
500 0.05 0.8674621491892098
500 0.1 0.8668966438128165
1000 0.01 0.867698457102168
1000 0.05 0.8674621491892098
1000 0.1 0.8668966438128165
1200 0.01 0.867698457102168
1200 0.05 0.8674621491892098
1200 0.1 0.8668966438128165
0.867698457102168


In [28]:
# See the best one is n_estimators=1000, learning_rate=0.01
# So train final model
model_fin = XGBClassifier(n_estimators=1000, learning_rate=0.01, n_jobs=-1, early_stopping_rounds=5, random_state=42)
model_fin.fit(X_train_prep, y_train,
             eval_set=[(X_valid_prep, y_valid)],
             verbose=False)
y_pred = model_fin.predict_proba(X_valid_prep)
score = roc_auc_score(y_valid, y_pred, average='macro')
print(score)

0.867698457102168


In [26]:
# Load the test data
X_test = pd.read_csv("test_set_features.csv")

# Initiate the output dataframe with id's
output = pd.DataFrame(X_test["respondent_id"])

# Preprocess the test data
X_test_prep = X_test.copy()

# Fill in missing values
for col in numerical_cols:
    X_test_prep[col].fillna(X_test_prep[col].mean(), inplace=True)

for col in nominal_cols:
    X_test_prep[col].fillna("Unknown", inplace=True)

for col in ordinal_cols:
    X_test_prep[col].fillna(X_test_prep[col].mode()[0], inplace=True)

# Sort by respondent_id
X_test_prep = X_test_prep.sort_values("respondent_id")

# Preprocess the ordinal columns
X_test_prep["age_group"] = pd.Categorical(X_test_prep["age_group"], categories=age_group_order, ordered=True)
X_test_prep["age_group"] = X_test_prep["age_group"].cat.codes

X_test_prep["education"] = pd.Categorical(X_test_prep["education"], categories=edu_order, ordered=True)
X_test_prep["education"] = X_test_prep["education"].cat.codes

X_test_prep["income_poverty"] = pd.Categorical(X_test_prep["income_poverty"], categories=inc_order, ordered=True)
X_test_prep["income_poverty"] = X_test_prep["income_poverty"].cat.codes

# One-hot encode the nominal columns
X_test_prep = pd.get_dummies(X_test_prep, columns=nominal_cols, dtype=int)
X_test_prep.head()

C:\Users\Marco van der Ploeg\AppData\Local\Temp\ipykernel_19988\3701370094.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_test_prep[col].fillna(X_test_prep[col].mean(), inplace=True)
C:\Users\Marco van der Ploeg\AppData\Local\Temp\ipykernel_19988\3701370094.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setti

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,employment_occupation_qxajmpny,employment_occupation_rcertsgn,employment_occupation_tfqavkke,employment_occupation_ukymxvdu,employment_occupation_uqqtjvyb,employment_occupation_vlluhbov,employment_occupation_xgwztkwe,employment_occupation_xqwwgdyp,employment_occupation_xtkaffoo,employment_occupation_xzmlyyjv
0,26707,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1,26708,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
2,26709,2.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3,26710,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,26711,3.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
# Make predictions
y_pred = model_fin.predict_proba(X_test_prep)
y_pred_h1n1 = y_pred[:, 0]
y_pred_seas = y_pred[:, 1]

# Add the predictions to the output dataframe
output["h1n1_vaccine"] = y_pred_h1n1
output["seasonal_vaccine"] = y_pred_seas
output.head()

,respondent_id,h1n1_vaccine,seasonal_vaccine
0,26707,0.129835,0.157189
1,26708,0.019725,0.037448
2,26709,0.169137,0.641654
3,26710,0.630137,0.856454
4,26711,0.203787,0.553883


In [30]:
# Save the output as csv
output.to_csv("submission.csv", index=False)

### Final note:

After submission, this result got a score of 0.8572.